In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch import autograd
# from elastic_weight_consolidation import ElasticWeightConsolidation

In [3]:
class ElasticWeightConsolidation:
    def __init__(self, model, crit, device, lr=0.001, weight=1000000):
        self.model = model.to(device)
        self.weight = weight
        self.crit = crit
        self.device = device
        self.optimizer = optim.Adam(self.model.parameters(), lr)

    def _update_mean_params(self):
        for param_name, param in self.model.named_parameters():
            _buff_param_name = param_name.replace('.', '__')
            self.model.register_buffer(_buff_param_name+'_estimated_mean', param.data.clone())

    def _update_fisher_params(self, current_ds, batch_size, num_batch):
        dl = DataLoader(current_ds, batch_size, shuffle=True)
        log_liklihoods = []
        for i, (input, target) in enumerate(dl):
            input = input.to(self.device)
            target = target.to(self.device)
            if i > num_batch:
                break
            output = F.log_softmax(self.model(input), dim=1)
            log_liklihoods.append(output[:, target])
        log_likelihood = torch.cat(log_liklihoods).mean()
        grad_log_liklihood = autograd.grad(log_likelihood, self.model.parameters())
        _buff_param_names = [param[0].replace('.', '__') for param in self.model.named_parameters()]
        for _buff_param_name, param in zip(_buff_param_names, grad_log_liklihood):
            self.model.register_buffer(_buff_param_name+'_estimated_fisher', param.data.clone() ** 2)

    def register_ewc_params(self, dataset, batch_size, num_batches):
        self._update_fisher_params(dataset, batch_size, num_batches)
        self._update_mean_params()

    def _compute_consolidation_loss(self, weight):
        try:
            losses = []
            for param_name, param in self.model.named_parameters():
                _buff_param_name = param_name.replace('.', '__')
                estimated_mean = getattr(self.model, '{}_estimated_mean'.format(_buff_param_name))
                estimated_fisher = getattr(self.model, '{}_estimated_fisher'.format(_buff_param_name))
                losses.append((estimated_fisher * (param - estimated_mean) ** 2).sum())
            return (weight / 2) * sum(losses)
        except AttributeError:
            return 0

    def forward_backward_update(self, input, target):
        input = input.to(self.device)
        target = target.to(self.device)
        output = self.model(input)
        loss = self._compute_consolidation_loss(self.weight) + self.crit(output, target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def save(self, filename):
        torch.save(self.model, filename)

    def load(self, filename):
        self.model = torch.load(filename)

In [4]:
def accu(model, dataloader, device):
    model = model.eval()    # .to(device)
    acc = 0
    for input, target in dataloader:
        o = model(input.to(device))
        acc += (o.argmax(dim=1).long() == target.to(device)).float().mean()
    return acc / len(dataloader)

In [5]:
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm

In [6]:
# os.environ['CUDA_VISIBLE_DEVICES'] = '2'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mnist_train = datasets.MNIST("../data", train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST("../data", train=False, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(mnist_train, batch_size = 100, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size = 100, shuffle=False)

In [7]:
class LinearLayer(nn.Module):
    def __init__(self, input_dim, output_dim, act='relu', use_bn=False):
        super(LinearLayer, self).__init__()
        self.use_bn = use_bn
        self.lin = nn.Linear(input_dim, output_dim)
        self.act = nn.ReLU() if act == 'relu' else act
        if use_bn:
            self.bn = nn.BatchNorm1d(output_dim)
    def forward(self, x):
        if self.use_bn:
            return self.bn(self.act(self.lin(x)))
        return self.act(self.lin(x))

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)


In [8]:
class BaseModel(nn.Module):
    def __init__(self, num_inputs, num_hidden, num_outputs):
        super(BaseModel, self).__init__()
        self.f1 = Flatten()
        self.lin1 = LinearLayer(num_inputs, num_hidden, use_bn=True)
        self.lin2 = LinearLayer(num_hidden, num_hidden, use_bn=True)
        self.lin3 = nn.Linear(num_hidden, num_outputs)
        
    def forward(self, x):
        return self.lin3(self.lin2(self.lin1(self.f1(x))))

In [9]:
crit = nn.CrossEntropyLoss()
# ewc = ElasticWeightConsolidation(BaseModel(28 * 28, 100, 10), crit=crit, lr=1e-4)
ewc = ElasticWeightConsolidation(BaseModel(28 * 28, 100, 10), crit=crit, lr=1e-4, device=device)

In [10]:
for _ in range(2):
    for input, target in tqdm(train_loader):
        ewc.forward_backward_update(input, target)

100%|██████████| 600/600 [00:14<00:00, 41.62it/s]


In [11]:
accu(ewc.model, test_loader, device)

tensor(0.9537, device='cuda:0')

In [12]:
ewc.register_ewc_params(mnist_train, 100, 300)

In [13]:
f_mnist_train = datasets.FashionMNIST("../data", train=True, download=True, transform=transforms.ToTensor())
f_mnist_test = datasets.FashionMNIST("../data", train=False, download=True, transform=transforms.ToTensor())
f_train_loader = DataLoader(f_mnist_train, batch_size = 100, shuffle=True)
f_test_loader = DataLoader(f_mnist_test, batch_size = 100, shuffle=False)

In [14]:
for _ in range(2):
    for input, target in tqdm(f_train_loader):
        ewc.forward_backward_update(input, target)

100%|██████████| 600/600 [00:20<00:00, 28.69it/s]


In [15]:
ewc.register_ewc_params(f_mnist_train, 100, 300)

In [16]:
accu(ewc.model, f_test_loader, device)

tensor(0.7977, device='cuda:0')

In [17]:
accu(ewc.model, test_loader, device)

tensor(0.9497, device='cuda:0')

In [25]:
for _ in range(1):
    for index, (input, target) in enumerate(f_test_loader):
        if index<1:
            print(ewc.model(input[0:4].to(device)))
            print(target[0:4])

for _ in range(1):
    for index, (input, target) in enumerate(test_loader):
        if index<1:
            print(ewc.model(input[0:4].to(device)))
            print(target[0:4])

tensor([[-2.2218, -0.7605, -0.7667, -1.1336, -0.4702,  1.9325, -2.6517,  2.0251,
         -0.3380,  2.3826],
        [-1.0768, -1.5262,  6.0297, -0.4847,  3.0193, -4.1753,  3.5122, -3.2434,
         -0.8368, -3.8094],
        [ 1.1982,  5.9452, -0.1854,  0.8599, -1.3951, -3.2498, -0.3939,  0.7331,
         -4.5451, -0.3555],
        [-0.7796,  6.7479, -0.5740,  2.0298, -2.2204, -2.9785,  0.7256, -0.5305,
         -4.2782, -0.1345]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([9, 2, 1, 1])
tensor([[ 0.9272, -0.7868,  1.1933,  0.5485, -0.9731, -1.1345, -1.2602,  6.5175,
         -2.0585,  0.1798],
        [-0.5558,  0.2415,  5.0362,  1.8427, -1.4949, -0.0082,  0.3034, -1.7884,
         -1.2401, -2.6832],
        [-0.7213,  5.8563, -0.1411, -1.2685, -1.0508,  0.3604,  0.2270, -0.6103,
         -1.7471, -0.3579],
        [ 5.9507, -0.6688, -0.8824, -0.9312, -0.1650, -0.1033,  1.3465, -0.2621,
         -1.3169, -0.2685]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([7, 2, 1, 0])
